1. Розібратись із прикладами нейромереж для класифікації датасету FashionMNIST (Dense слої)
2. Розібратись із прикладами нейромереж для класифікації датасету FashionMNIST (Convolutions слої)

## INIT BLOCK

### import requirements

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten
from tensorflow.python.keras.utils.np_utils import to_categorical
from tensorflow.python.keras.metrics import Precision, Recall, AUC
from tensorflow.python.keras.models import Model

### CONSTANTS

In [ ]:
NUMBER_OF_TRAIN_DATA: int = 60000
NUMBER_OF_TEST_DATA: int = 10000

IMG_ROWS: int = 28
IMG_COLS: int = 28

NUMBER_OF_CLASSES: int = 10
COLOR_DIVIDER: int = 255

### DATA COLLECTION

In [ ]:
x_train: np.ndarray
y_train: np.ndarray
x_test: np.ndarray
y_test: np.ndarray

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

assert x_train.shape == (NUMBER_OF_TRAIN_DATA, IMG_ROWS, IMG_COLS)
assert x_test.shape == (NUMBER_OF_TEST_DATA, IMG_ROWS, IMG_COLS)
assert y_train.shape == (NUMBER_OF_TRAIN_DATA,)
assert y_test.shape == (NUMBER_OF_TEST_DATA,)

## DATA PREPARING

In [ ]:
x_train: np.ndarray = x_train.reshape(-1, IMG_ROWS * IMG_COLS) / COLOR_DIVIDER
x_test: np.ndarray = x_test.reshape(-1, IMG_ROWS * IMG_COLS) / COLOR_DIVIDER

In [ ]:
y_train: np.ndarray = to_categorical(y=y_train, num_classes=NUMBER_OF_CLASSES, dtype=int)
y_test: np.ndarray = to_categorical(y=y_test, num_classes=NUMBER_OF_CLASSES, dtype=int)

In [ ]:
def train_model(model, x_train=x_train, x_test=x_test):
    model.compile(
        optimizer="nadam",
        loss="categorical_crossentropy",
        metrics=['accuracy', Precision(), Recall(), AUC()]
    )

    print(model.summary(), end="\n\n")

    model.fit(
        y=y_train,
        x=x_train,
        epochs=10,
        batch_size=128,
        validation_split=0.2,
        verbose=False
    )

    loss, accuracy, precision, recall, auc = model.evaluate(x=x_test, y=y_test)

    print(f"Loss: {loss:.3f}")
    print(f"Accuracy: {accuracy:.3f}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"AUC: {auc:.3f}")

# DENSE LAYERS

## Sequential model

Model Init

In [ ]:
seq_model: Sequential = Sequential(
    layers=[
        Dense(units=128, activation="relu", input_shape=(IMG_ROWS * IMG_COLS,)),
        Dense(units=64, activation="relu"),
        Dense(units=NUMBER_OF_CLASSES, activation="softmax")
    ]
)

Model Fit/Model metrics

In [ ]:
train_model(model=seq_model)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 10)                650       
                                                                 
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
_________________________________________________________________
None

313/313 [==============================] - 1s 4ms/step - loss: 0.3416 - accuracy: 0.8801 - precision: 0.9077 - recall: 0.8587 - auc: 0.9927
Loss: 0.342
Accuracy: 0.880
Precision: 0.908
Recall: 0.859
AUC: 0.993


## Functional API style

Model init

In [ ]:
inputs: Input = Input(shape=(IMG_ROWS * IMG_COLS,))

hidden_layer_1: Dense = Dense(units=128, activation="relu")(inputs)
hidden_layer_2: Dense = Dense(units=64, activation="relu")(hidden_layer_1)
output_layer: Dense = Dense(units=NUMBER_OF_CLASSES, activation="softmax")(hidden_layer_2)

api_style_model = Model(inputs=inputs, outputs=output_layer)

Model Fit/Model metrics

In [ ]:
train_model(model=api_style_model)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
_________________________________________________________________
None

313/313 [==============================] - 1s 3ms/step - loss: 0.3416 - accuracy: 0.8800 - precision_1: 0.8973 - recall_1: 0.8670 - auc_1: 0.9911
Loss: 0.342
Accuracy: 0.880
Precision: 0.897
Recall: 0.867
AUC: 0.991


# Convolutions Layers

Data preparing

In [ ]:
x_train: np.ndarray
y_train: np.ndarray
x_test: np.ndarray
y_test: np.ndarray

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

assert x_train.shape == (NUMBER_OF_TRAIN_DATA, IMG_ROWS, IMG_COLS)
assert x_test.shape == (NUMBER_OF_TEST_DATA, IMG_ROWS, IMG_COLS)
assert y_train.shape == (NUMBER_OF_TRAIN_DATA,)
assert y_test.shape == (NUMBER_OF_TEST_DATA,)

x_train: np.ndarray = x_train.reshape(-1, IMG_ROWS, IMG_COLS, 1) / COLOR_DIVIDER
x_test: np.ndarray = x_test.reshape(-1, IMG_ROWS, IMG_COLS, 1) / COLOR_DIVIDER

y_train: np.ndarray = to_categorical(y=y_train, num_classes=NUMBER_OF_CLASSES, dtype=int)
y_test: np.ndarray = to_categorical(y=y_test, num_classes=NUMBER_OF_CLASSES, dtype=int)

## Functional API style

Model init

In [ ]:
inputs: Input = Input(shape=(IMG_ROWS, IMG_COLS, 1))

conv_layer_1: Conv2D = Conv2D(32, (3, 3), activation="relu")(inputs)
maxpool_layer_1: MaxPooling2D = MaxPooling2D((2, 2))(conv_layer_1)

conv_layer_2: Conv2D = Conv2D(64, (3, 3), activation="relu")(maxpool_layer_1)
maxpool_layer_2: MaxPooling2D = MaxPooling2D((2, 2))(conv_layer_2)

conv_layer_3: Conv2D = Conv2D(128, (3, 3), activation="relu")(maxpool_layer_2)
flatten_layer: Flatten = Flatten()(conv_layer_3)

output_layer: Dense = Dense(units=10, activation="softmax")(flatten_layer)

convolution_model: Model = Model(inputs=inputs, outputs=output_layer)

Model Fit/Model metrics

In [ ]:
train_model(model=convolution_model, x_test=x_test, x_train=x_train)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 104,202
Trainable params: 104,202
Non-trainable params: 0
_________________________________________________________________
None

313/313 [==============================] - 7s 21ms/step - loss: 0.2647 - accuracy: 0.9048 - precision_4: 0.9189 - recall_4: 0.8962 - auc_4: 0.9945
Loss: 0.265
Accuracy: 0.905
Precision: 0.919
Recall: 0.896
AUC: 0.995


## Sequential model

Model init

In [ ]:
seq_convolution_model: Sequential = Sequential(
    layers=[
        Conv2D(32, (3, 3), activation="relu", input_shape=(IMG_ROWS, IMG_COLS, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation="relu"),
        Flatten(),
        Dense(units=64, activation="relu"),
        Dense(units=10, activation="softmax")
    ]
)

Model Fit/Model metrics

In [ ]:
train_model(model=seq_convolution_model, x_test=x_test, x_train=x_train)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_13 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 3, 3, 64)          36928     
                                                                 
 flatten_4 (Flatten)         (None, 576)              

# Висновок

Зробив класифікацію датасету Fashion MNIST

1.   Із слоями Dense, у вигляді функціонального та послідовного стилів. Точність цих моделей становить приблизно 88%.


2.   Із слоями Convolution, у вигляді функціонального та послідовного стилів. Точність цих моделей становить приблизно 90%, що на 2% більше за попередні.
